# Ford - Fulkerson Network Maximum Flow via DFS

The implementation of Ford - Fulkerson algorithm shown below utilizes the Depth First Search for finding augmenting paths during graph traversal.

**Time complexity**: 
- <font color="orange" size="3"><b>O(fE)</b></font>, where *f* is the maxumum flow and *E* is the number of edges.

The Ford-Fulkerson algorithm and the network maximum flow problem have various real-world applications. Here are some examples:

- **Transportation Networks**: The maximum flow problem can be applied to transportation networks, such as traffic flow in road networks or the flow of goods in supply chain systems. By determining the maximum flow between nodes, we can optimize the utilization of resources and identify potential bottlenecks.

- **Telecommunications**: Network flow algorithms are used in telecommunications to optimize data transmission across a network, such as routing data packets through routers or allocating bandwidth in communication networks. Maximizing the flow can help improve the efficiency and reliability of data transmission.

- **Water Distribution**: The maximum flow algorithm can be applied to optimize the distribution of water in a network of pipes or canals. By determining the maximum flow, we can ensure that water is efficiently allocated to different regions, taking into account constraints such as pipe capacities and demand at different locations.

- **Energy Networks**: The maximum flow problem is relevant to energy distribution networks, such as electrical power grids. It can be used to optimize the flow of electricity, considering factors like power generation, transmission line capacities, and consumer demands. This helps in managing the load balance and preventing overloads.

- **Image Segmentation**: In computer vision, the max-flow/min-cut algorithm (a related concept) is used for image segmentation. It helps to separate an image into meaningful regions by finding the optimal cut that separates foreground and background pixels based on the concept of maximum flow.

The network maximum flow problem involves determining the maximum flow that can be sent through a network from a source to a sink, considering capacity limits on edges. It's represented by a directed graph where nodes are entities and edges are connections. The goal is to find the most efficient flow route while respecting capacity constraints.

The Ford-Fulkerson algorithm iteratively finds paths from source to sink that can accommodate more flow. By updating flow values along these paths, it increases the overall flow until no more paths exist. The algorithm stops when adding more flow would violate capacity limits, yielding the maximum flow value and flow distribution.

This problem and algorithm have practical applications in optimizing transportation networks, telecommunications, water distribution, energy grids, and image segmentation. They enable resource allocation, efficiency improvement, and informed decision-making in various scenarios.

In [2]:
class ford_fulkerson_example:
    
    class Edge:
        '''
        Represents an edge in the graph with its capacity, flow, and residual edge.
        '''
        def __init__(self, frm, to, capacity):
            self.frm = frm
            self.to = to
            self.residual = None
            self.capacity = capacity
            self.flow = 0
        
        def is_residual(self):
            '''
            Checks if the edge is a residual edge (capacity is zero).
            '''
            return self.capacity == 0
        
        def remaining_capacity(self):
            '''
            Calculates the remaining capacity of the edge.
            '''
            return self.capacity - self.flow
        
        def augment(self, bottleneck):
            '''
            Increases the flow of the edge by the given bottleneck value
            and decreases the flow of the residual edge by the same value.
            '''
            self.flow += bottleneck
            self.residual.flow -= bottleneck
     
        def __repr__(self):
            '''
            Returns a string representation of the edge.
            '''
            return str((self.frm, self.to, self.capacity))
    
    class networkflow_solver_base:
        '''
        Base class for a network maximum flow solver.
        '''
        def __init__(self, n, s, t):
            '''
            Initializes the solver with the number of nodes, source node index (s),
            and sink node index (t).
            '''
            self.n = n
            self.s = s
            self.t = t
            self.visited_token = 1
            self.visited = [0] * self.n
            self.solved = False
            self.max_flow = 0
            self.initialize_empty_flow_graph()
            
        def add_edge(self, frm, to, capacity):
            '''
            Adds an edge to the graph with the given source, destination, and capacity.
            '''
            if capacity <= 0:
                raise ValueError(f'Illegal capacity for forward edge: {capacity} <= 0')
            e1 = ford_fulkerson_example.Edge(frm, to, capacity)
            e2 = ford_fulkerson_example.Edge(to, frm, 0)
            e1.residual = e2
            e2.residual = e1
            self.graph[frm].append(e1)
            self.graph[to].append(e2)
            
        def initialize_empty_flow_graph(self):
            '''
            Initializes an empty graph with no edges.
            '''
            self.graph = [[] for _ in range(self.n)]
        
        def get_graph(self):
            '''
            Returns the flow graph representation.
            '''
            self.execute()
            return self.graph
    
        def get_max_flow(self):
            '''
            Returns the maximum flow value.
            '''
            self.execute()
            return self.max_flow
        
        def execute(self):
            '''
            Executes the flow solver if it hasn't been solved before.
            '''
            if self.solved:
                return
            self.solved = True
            self.solve()
            
        def solve(self):
            '''
            Abstract method to be implemented by subclasses.
            '''
            pass
        
    class ford_fulkerson_dfs_solver(networkflow_solver_base):
        '''
        Subclass of networkflow_solver_base that implements the Ford-Fulkerson algorithm using DFS.
        '''
        def __init__(self, n, s, t):
            super().__init__(n, s, t)
            
        def visualize_flow(self):
            '''
            Visualizes the network graph and its maximum flow.
            '''
            self.execute()

            # Print the network graph with flow values
            print("Network Graph:")
            for node in range(self.n):
                edges = self.graph[node]
                for edge in edges:
                    print(f"({edge.frm}) -- [ {edge.flow}/{edge.capacity} ]--> ({edge.to}) | Is residual: {edge.is_residual()}")

            # Print the maximum flow value
            print("Maximum Flow:", self.max_flow)    
            
        def solve(self):
            '''
            Solves the maximum flow problem using the Ford-Fulkerson algorithm with DFS.
            '''
            flow = self.dfs(self.s, float('inf'))
            while flow != 0:
                self.visited_token += 1
                self.max_flow += flow
                flow = self.dfs(self.s, float('inf'))
        
        def dfs(self, node, flow_):
            '''
            Performs a depth-first search to find augmenting paths and calculate flow values.
            '''
            if node == self.t:
                return flow_
        
            self.visited[node] = self.visited_token
        
            edges = self.graph[node]
            for edge in edges:
                if edge.remaining_capacity() > 0 and self.visited[edge.to] != self.visited_token:
                    bottleneck = self.dfs(edge.to, min(flow_, edge.remaining_capacity()))
                    if bottleneck > 0:
                        edge.augment(bottleneck)
                        return bottleneck
            return 0

In [3]:
n = 12     # Number of nodes in the graph
s = n - 2  # Index of the 'source' node
t = n - 1  # Index of the 'sink' node

# Initializing solver class object
solver = ford_fulkerson_example.ford_fulkerson_dfs_solver(n, s, t)

# Initializing edges from the source
solver.add_edge(solver.s, 0, 10)
solver.add_edge(solver.s, 1, 5)
solver.add_edge(solver.s, 2, 10)

# Initializing middle edges
solver.add_edge(0, 3, 10)
solver.add_edge(1, 2, 10)
solver.add_edge(2, 5, 15)
solver.add_edge(3, 1, 2)
solver.add_edge(3, 6, 15)
solver.add_edge(4, 1, 15)
solver.add_edge(4, 3, 3)
solver.add_edge(5, 4, 4)
solver.add_edge(5, 8, 10)
solver.add_edge(6, 7, 10)
solver.add_edge(7, 4, 10)
solver.add_edge(7, 5, 7)

# Initializing edges to the sink
solver.add_edge(6, solver.t, 15)
solver.add_edge(8, solver.t, 10)

print(f'Maximum flow sustainable on a graph: {solver.get_max_flow()}')

Maximum flow sustainable on a graph: 23


In [4]:
solver.visualize_flow()

Network Graph:
(0) -- [ -10/0 ]--> (10) | Is residual: True
(0) -- [ 10/10 ]--> (3) | Is residual: False
(1) -- [ -5/0 ]--> (10) | Is residual: True
(1) -- [ 5/10 ]--> (2) | Is residual: False
(1) -- [ 0/0 ]--> (3) | Is residual: True
(1) -- [ 0/0 ]--> (4) | Is residual: True
(2) -- [ -8/0 ]--> (10) | Is residual: True
(2) -- [ -5/0 ]--> (1) | Is residual: True
(2) -- [ 13/15 ]--> (5) | Is residual: False
(3) -- [ -10/0 ]--> (0) | Is residual: True
(3) -- [ 0/2 ]--> (1) | Is residual: False
(3) -- [ 13/15 ]--> (6) | Is residual: False
(3) -- [ -3/0 ]--> (4) | Is residual: True
(4) -- [ 0/15 ]--> (1) | Is residual: False
(4) -- [ 3/3 ]--> (3) | Is residual: False
(4) -- [ -3/0 ]--> (5) | Is residual: True
(4) -- [ 0/0 ]--> (7) | Is residual: True
(5) -- [ -13/0 ]--> (2) | Is residual: True
(5) -- [ 3/4 ]--> (4) | Is residual: False
(5) -- [ 10/10 ]--> (8) | Is residual: False
(5) -- [ 0/0 ]--> (7) | Is residual: True
(6) -- [ -13/0 ]--> (3) | Is residual: True
(6) -- [ 0/10 ]--> (7) | I